## Final Project - SQL

#### Task

- Find the number of books released after January 1, 2000.
- Find the number of user reviews and the average rating for each book.
- Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).
- Identify the author with the highest average book rating (look only at books with at least 50 ratings).
- Find the average number of text reviews among users who rated more than 50 books.

#### Instructions for completing the task

- Describe the goals of the study.
- Study the tables (print the first rows).
- Make an SQL query for each of the tasks.
- Output the results of each query in the Notebook.
- Describe your conclusions for each of the tasks.

In [1]:
# import libraries

import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
books = pd.io.sql.read_sql('SELECT * from books', con = engine)
authors = pd.io.sql.read_sql('SELECT * from authors', con = engine)
publishers = pd.io.sql.read_sql('SELECT * from publishers', con = engine)
ratings = pd.io.sql.read_sql('SELECT * from ratings', con = engine)
reviews = pd.io.sql.read_sql('SELECT * from reviews', con = engine)

In [4]:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [9]:
books.duplicated().sum()

0

In [10]:
authors.duplicated().sum()

0

In [11]:
publishers.duplicated().sum()

0

In [12]:
ratings.duplicated().sum()

0

In [13]:
reviews.duplicated().sum()

0

In [14]:
books.isna().sum()

book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64

In [15]:
authors.isna().sum()

author_id    0
author       0
dtype: int64

In [16]:
publishers.isna().sum()

publisher_id    0
publisher       0
dtype: int64

In [17]:
ratings.isna().sum()

rating_id    0
book_id      0
username     0
rating       0
dtype: int64

In [18]:
reviews.isna().sum()

review_id    0
book_id      0
username     0
text         0
dtype: int64

In [19]:
# Find the number of books released after January 1, 2000.

books_after_2001_01_01_query = '''
SELECT 
    COUNT(*) 
FROM books 
WHERE 
    CAST(publication_date AS date) > '2000-01-01';
'''
books_after_2001_01_01 = pd.io.sql.read_sql(books_after_2001_01_01_query, con = engine)
books_after_2001_01_01

,count
0,819


There are only 819 books released after January 1, 2000, so perhaps this should comprise the majority of the books sold, and only a smaller percentage from older books, only those with highest ratings.

In [20]:
# Find the number of user reviews and the average rating for each book.

user_reviews_avg_rating_query = '''
SELECT 
    DISTINCT b.title as book,
    COUNT (DISTINCT r.review_id) AS reviews_number, 
    AVG(i.rating) AS avg_rating
FROM ratings 
as i INNER JOIN reviews AS r ON i.book_id = r.book_id
INNER JOIN books AS b on b.book_id = r.book_id
GROUP BY book
ORDER BY reviews_number DESC;
'''

user_reviews_avg_rating_query = pd.io.sql.read_sql(user_reviews_avg_rating_query, con = engine)
user_reviews_avg_rating_query

,book,reviews_number,avg_rating
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,Eat Pray Love,6,3.395833
3,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
4,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
...,...,...,...
988,The Social Contract,1,3.500000
989,To Green Angel Tower (Memory Sorrow and Thor...,1,4.500000
990,Tsubasa: RESERVoir CHRoNiCLE Vol. 1,1,2.500000
991,Wicked: The Grimmerie,1,3.500000


In [21]:
# Identify the publisher that has released the greatest number of books with more than 50 pages 
# (this will help you exclude brochures and similar publications from your analysis).

greatest_publisher_query = '''
SELECT p.publisher, 
    COUNT(DISTINCT b.book_id) as total_books 
FROM publishers 
AS p INNER JOIN books AS b ON p.publisher_id = b.publisher_id 
WHERE 
    b.num_pages>50 
GROUP BY p.publisher_id 
ORDER BY total_books DESC LIMIT 1;
'''

greatest_publisher_query = pd.io.sql.read_sql(greatest_publisher_query, con = engine)
greatest_publisher_query

,publisher,total_books
0,Penguin Books,42


In [22]:
# Identify the author with the highest average book rating (look only at books with at least 50 ratings).

highest_avg_rating_author_query = '''
SELECT a.author, 
    a.author_id, 
    AVG(r.rating) as avg_rating 
FROM authors 
AS a INNER JOIN books AS k ON a.author_id = k.author_id 
INNER JOIN ratings AS r ON r.book_id=k.book_id 
WHERE k.book_id IN 
    (SELECT b.book_id 
    FROM books 
    AS b INNER JOIN ratings AS s ON b.book_id = s.book_id 
    GROUP BY b.book_id HAVING COUNT(DISTINCT s.rating_id)>50) 
GROUP BY a.author_id 
ORDER BY avg_rating DESC LIMIT 1;
'''

highest_avg_rating_author_query = pd.io.sql.read_sql(highest_avg_rating_author_query, con = engine)
highest_avg_rating_author_query

,author,author_id,avg_rating
0,J.K. Rowling/Mary GrandPré,236,4.287097


In [23]:
# Find the average number of text reviews among users who rated more than 50 books.

avg_text_reviews_query = '''
WITH i AS 
    (SELECT username AS username, 
    COUNT(rating_id) AS books_rated 
    FROM ratings 
    GROUP BY username 
    HAVING COUNT(rating_id)>50 
    ORDER BY books_rated DESC), 
b AS (SELECT username, 
    COUNT(review_id) AS total_reviews 
    FROM reviews GROUP BY username) 
SELECT AVG(b.total_reviews) AS avg_text_reviews 
FROM i INNER JOIN b ON i.username = b.username;
'''

avg_text_reviews_query = pd.io.sql.read_sql(avg_text_reviews_query, con = engine)
avg_text_reviews_query

,avg_text_reviews
0,24.333333


Users who rated more than 50 books have on average 24 text reviews. This means that we are getting more number ratings than text reviews. We can focus on prioritizing books with numbered ratings, and not pay so much attention to text reviews.

#### Conclusions

**The business should prioritize:**

- Books from the publisher Penguin Books
- Books with ratings larger or equal to the average rating of all books (3.936791)
- Books of author 236 - J.K. Rowling/Mary GrandPré

**Books' age:**
- There are only 819 books released after January 1, 2000, so perhaps this should comprise the majority of the books sold, and only a smaller percentage from older books, only those with highest ratings.

**Books' ratings:**
- It would be great to sell mostly books with at least 24 text reviews, as those are probably the most popular ones.
- Users who rated more than 50 books have on average 24 text reviews. This means that we are getting more number ratings than text reviews. We can focus on prioritizing books with numbered ratings, and not pay so much attention to text reviews.